In [7]:
import networkx as nx
import numpy as np
import heapq
import unittest
from collections import deque
import matplotlib.pyplot as plt
from queue import Queue

In [8]:
# Adjacency list of our graph
small_graph = {
    "a": {"b" : 85, "c": 95, "d": 125},
    "b": {"a": 85, "c": 70, "e": 200, "f": 290},
    "c": {"a": 95, "b": 70, "d": 100},
    "d": {"a": 125, "c": 100, "e": 175},
    "e": {"b": 200, "d": 175, "f": 110, "g": 20},
    "f": {"b": 280, "e": 110, "g": 100},
    "g": {"e": 20, "f": 100}
}

# SumOfPaths
---

In [9]:
# Single Source Shortest Paths Implementation
def dijkstra(source, graph):
    if source not in graph:
        return None
    visited = set()
    cost_path = {n: float("inf") for n in graph}
    cost_path[source] = (0)
    while len(visited) != len(graph):
        min_node = None
        min_val = float("inf")
        for n in cost_path:
            if n not in visited:
                if cost_path[n] < min_val:
                    min_val = cost_path[n]
                    min_node = n
        visited.add(min_node)
        for n in graph[min_node]:
            if n not in visited:
                if cost_path[min_node] + graph[min_node][n] < cost_path[n]:
                    cost_path[n] = (cost_path[min_node] + graph[min_node][n])
    return cost_path

# SumOfPaths: Find the node with the short summed cost from every source node. This approach IS guaranteed to find
# the optimal meeting point, defined as the point that would required the least total travel from all sources, for
# any number of sources (N) in a weighted, connected graph. Note that it expands a full SSSP from every source node.
def meetSum(sources, graph):
    # find distance from sources to all points
    dists_to_sources = [dijkstra(source, graph) for source in sources]
    print("\n# of Nodes Expanded: %d" % (len(sources) * len(graph)))
    print("----------------------------")
    # sum distances from all sources to every node
    total_dists = {g: 0 for g in graph}
    for dist_dict in dists_to_sources:
        for k in dist_dict:
            total_dists[k] += dist_dict[k] 
            
    # choose the node with minimum distance
    meet_point = min(total_dists, key=total_dists.get)
    print("Meeting point: " + meet_point)
    for i, source in enumerate(sources):
        print("Distance from %s: %d" % (source, dists_to_sources[i][meet_point]))
    print("Total distance traveled:", total_dists[meet_point])
    return (meet_point, total_dists[meet_point])

### Running Examples:

In [10]:
# Running SumOfPaths on 6 examples
meetSum(["a", "d", "e"], small_graph)
meetSum(["a", "e", "f"], small_graph)
meetSum(["a", "c", "f"], small_graph)
meetSum(["c", "g", "d"], small_graph)
meetSum(["e", "f", "g"], small_graph)
meetSum(["a", "b", "c", "d", "f"], small_graph)


# of Nodes Expanded: 21
----------------------------
Meeting point: d
Distance from a: 125
Distance from d: 0
Distance from e: 175
Total distance traveled: 300

# of Nodes Expanded: 21
----------------------------
Meeting point: e
Distance from a: 285
Distance from e: 0
Distance from f: 110
Total distance traveled: 395

# of Nodes Expanded: 21
----------------------------
Meeting point: b
Distance from a: 85
Distance from c: 70
Distance from f: 280
Total distance traveled: 435

# of Nodes Expanded: 21
----------------------------
Meeting point: d
Distance from c: 100
Distance from g: 195
Distance from d: 0
Total distance traveled: 295

# of Nodes Expanded: 21
----------------------------
Meeting point: g
Distance from e: 20
Distance from f: 100
Distance from g: 0
Total distance traveled: 120

# of Nodes Expanded: 35
----------------------------
Meeting point: b
Distance from a: 85
Distance from b: 0
Distance from c: 70
Distance from d: 170
Distance from f: 280
Total distance traveled:

('b', 605)

### Results:

__Output:__ SumOfPaths found the optimal meeting point and the optimal distance traveled in every example. 

__Performance:__ SumOfPaths had to exapand M * N nodes for M source nodes and N total nodes

# Method 1: BFS
___

In [11]:
# Method 1: BFS from every source node, terminating when paths from all sources collide. This will find the optimal
# meeting point on unweighted (or unit edge) graphs ONLY. It is NOT guaranteed to find the optimal meeting point on 
# weighted graphs, though it may in some instances.

def meetBFS(sources, graph):
    frontiers = {}
    paths = {}
    nodes_expanded = 0
    for source in sources:
        frontiers[source] = Queue()
        frontiers[source].put(source)
        paths[source] = {source: (0, [source])}
        
    while any(frontiers.values()):
        for source in frontiers.keys():
            node = frontiers[source].get()
            nodes_expanded += 1
            found = True
            for k_path in paths.keys():
                if k_path != source and node not in paths[k_path]:
                    found = False
            if found:
                print("\n# of Nodes Expanded: %d" % (nodes_expanded))
                print("----------------------------")
                print("Meeting point:", node)
                total = 0
                for s in paths.keys():
                    print("Path from %s to %s: " % (s, node), paths[s][node])
                    total += paths[s][node][0]
                print("Total distance traveled:", total)
                return node
            else:
                for n in graph[node]:
                    if n not in paths[source]:
                        paths[source][n] = (paths[source][node][0] + graph[node][n], paths[source][node][1] + [n])
                        frontiers[source].put(n)
                        
        for key in [k for k in frontiers.keys()]:
            if frontiers[key].empty():
                frontiers.pop(key)
    return None   
    

### Running Examples:

In [12]:
# Running BFS method on 6 examples
meetBFS(["a", "d", "e"], small_graph)
meetBFS(["a", "f", "e"], small_graph)
meetBFS(["a", "c", "f"], small_graph)
meetBFS(["c", "g", "d"], small_graph)
meetBFS(["e", "f", "g"], small_graph)
meetBFS(["a", "b", "c", "d", "f"], small_graph)


# of Nodes Expanded: 6
----------------------------
Meeting point: b
Path from a to b:  (85, ['a', 'b'])
Path from d to b:  (210, ['d', 'a', 'b'])
Path from e to b:  (200, ['e', 'b'])
Total distance traveled: 495

# of Nodes Expanded: 4
----------------------------
Meeting point: b
Path from a to b:  (85, ['a', 'b'])
Path from f to b:  (280, ['f', 'b'])
Path from e to b:  (200, ['e', 'b'])
Total distance traveled: 565

# of Nodes Expanded: 4
----------------------------
Meeting point: b
Path from a to b:  (85, ['a', 'b'])
Path from c to b:  (70, ['c', 'b'])
Path from f to b:  (280, ['f', 'b'])
Total distance traveled: 435

# of Nodes Expanded: 7
----------------------------
Meeting point: b
Path from c to b:  (70, ['c', 'b'])
Path from g to b:  (220, ['g', 'e', 'b'])
Path from d to b:  (210, ['d', 'a', 'b'])
Total distance traveled: 500

# of Nodes Expanded: 3
----------------------------
Meeting point: g
Path from e to g:  (20, ['e', 'g'])
Path from f to g:  (100, ['f', 'g'])
Path fr

'b'

### Results:

__Output:__ BFS approach found the optimal meeting point and the optimal distance in only 2 of the 6 examples, which were just by happenstance. 

__Performance:__ BFS approach expanded much fewer nodes than SumOfPaths, but that doesn't mean much when it doesn't even find the optimal meeting point

# Method 2: Dijkstra
___

In [13]:
# Method 2: Dijkstra from every source node, terminating when paths from all sources collide. Again, this is 
# NOT guaranteed to find the optimal meeting point, though it can in some instances.

def meetDijkstra(sources, graph):
    visited = {}
    costs = {}
    paths = {}
    nodes_expanded = 0
    for source in sources:
        visited[source] = set()
        costs[source] = {n: float("inf") for n in graph}
        costs[source][source] = 0
        paths[source] = {source: [source]}
    while (True):
        for source in sources:
            min_node = None
            min_val = float("inf")
            for n in costs[source]:
                if n not in visited[source]:
                    if costs[source][n] < min_val:
                        min_node = n
            visited[source].add(min_node)
            nodes_expanded += 1
            found = True
            for k in sources:
                if k != source and min_node not in visited[k]:
                    found = False
            if found:
                print("\n# of Nodes Expanded: %d" % (nodes_expanded))
                print("----------------------------")
                total = 0
                print("Meeting Point:", min_node)
                for s in paths.keys():
                    print("Path from %s to %s: " % (s, min_node), (costs[s][min_node], paths[s][min_node]))
                    total += costs[s][min_node]
                print("Total distance traveled:", total)
                return min_node
            for n in graph[min_node]:
                if n not in visited[source]:
                    if costs[source][min_node] + graph[min_node][n] < costs[source][n]:
                        costs[source][n] = (costs[source][min_node] + graph[min_node][n])
                        paths[source][n] = paths[source][min_node] + [n]
            
    

### Running Examples:

In [14]:
# Running Dijkstra method on 6 examples
meetDijkstra(["a", "d", "e"], small_graph)
meetDijkstra(["a", "f", "e"], small_graph)
meetDijkstra(["a", "c", "f"], small_graph)
meetDijkstra(["c", "g", "d"], small_graph)
meetDijkstra(["e", "f", "g"], small_graph)
meetDijkstra(["a", "b", "c", "d", "f"], small_graph)


# of Nodes Expanded: 7
----------------------------
Meeting Point: e
Path from a to e:  (300, ['a', 'd', 'e'])
Path from d to e:  (175, ['d', 'e'])
Path from e to e:  (0, ['e'])
Total distance traveled: 475

# of Nodes Expanded: 8
----------------------------
Meeting Point: e
Path from a to e:  (300, ['a', 'd', 'e'])
Path from f to e:  (110, ['f', 'e'])
Path from e to e:  (0, ['e'])
Total distance traveled: 410

# of Nodes Expanded: 9
----------------------------
Meeting Point: e
Path from a to e:  (300, ['a', 'd', 'e'])
Path from c to e:  (275, ['c', 'd', 'e'])
Path from f to e:  (110, ['f', 'e'])
Total distance traveled: 685

# of Nodes Expanded: 8
----------------------------
Meeting Point: e
Path from c to e:  (275, ['c', 'd', 'e'])
Path from g to e:  (20, ['g', 'e'])
Path from d to e:  (175, ['d', 'e'])
Total distance traveled: 470

# of Nodes Expanded: 5
----------------------------
Meeting Point: g
Path from e to g:  (20, ['e', 'g'])
Path from f to g:  (100, ['f', 'g'])
Path fr

'e'

### Results:

__Output:__ Dijkstras approach found the optimal meeting point and the optimal distance in only 1 of the 6 examples, which was just by happenstance. 

__Performance:__ Dijkstras approach expanded much fewer nodes than SumOfPaths, but not as few as BFS, and again that doesn't mean much when it doesn't even find the optimal meeting point.

# Bidirectional A* Search
___

In [15]:
def path(previous, s):
    '''
    `previous` is a dictionary chaining together the predecessor state that led to each state
    `s` will be None for the initial state
    otherwise, start from the last state `s` and recursively trace `previous` back to the initial state,
    constructing a list of states visited as we go
    '''
    if s is None:
        return []
    else:
        return path(previous, previous[s])+[s]

def pathcost(path, step_costs):
    '''
    add up the step costs along a path, which is assumed to be a list output from the `path` function above
    '''
    cost = 0
    for s in range(len(path)-1):
        cost += step_costs[path[s]][path[s+1]]
    return cost

map_distances = dict(
    chi=dict(det=283, cle=345, ind=182),
    cle=dict(chi=345, det=169, col=144, pit=134, buf=189),
    ind=dict(chi=182, col=176),
    col=dict(ind=176, cle=144, pit=185),
    det=dict(chi=283, cle=169, buf=256),
    buf=dict(det=256, cle=189, pit=215, syr=150),
    pit=dict(col=185, cle=134, buf=215, phi=305, bal=247),
    syr=dict(buf=150, phi=253, new=254, bos=312),
    bal=dict(phi=101, pit=247),
    phi=dict(pit=305, bal=101, syr=253, new=97),
    new=dict(syr=254, phi=97, bos=215, pro=181),
    pro=dict(bos=50, new=181),
    bos=dict(pro=50, new=215, syr=312, por=107),
    por=dict(bos=107))

forward_providence = dict(
    chi=833,
    cle=531,
    ind=782,
    col=618,
    det=596,
    buf=385,
    pit=458,
    syr=253,
    bal=325,
    phi=236,
    new=157,
    pro=0,
    bos=38,
    por=136)

backward_providence = dict(
    chi=0,
    cle=202,
    ind=75,
    col=215,
    det=350,
    buf=500,
    pit=450,
    syr=635,
    bal=575,
    phi=600,
    new=675,
    pro=833,
    bos=850,
    por=915)
# Solution:

def forward_heuristic_sld_providence(state):
    return forward_providence[state]
def backward_heuristic_sld_providence(state):
    return backward_providence[state]

class Frontier_PQ:
    ''' frontier class for uniform search, ordered by path cost '''
    def __init__(self,start,cost):
        self.states = {}
        self.q = []
        self.add(start,cost)
    def add(self,state,cost):
        heapq.heappush(self.q,(cost,state))
        self.states[state]=cost
    def pop(self):
        (cost,state) = heapq.heappop(self.q)
        self.states.pop(state)
        return (cost,state)
    def replace(self,state,cost):
        if(self.states[state] > cost):
            self.states[state] = cost
        for i,(currentCost,currentState) in enumerate(self.q):
            if (currentState == state and currentCost > cost):
                self.q[i] = (cost,state)
                heapq.heapify(self.q)
                return
    def contains(self, node):
        return node in self.states

def astar_search(start, goal, state_graph, forwardHeuristic, backwardHeuristic,return_cost=False, return_nexp=False):
    '''A* search from `start` to `goal`
    start = initial state
    goal = goal state
    heuristic = function for estimated cost to goal (function name)
    return_cost = logical (True/False) for whether or not to return the total path cost
    return_nexp = logical (True/False) for whether or not to return the number of nodes expanded
    '''
    frontStep = 0
    backStep = 0
    frontFrontier = Frontier_PQ(start,0)
    backFrontier = Frontier_PQ(goal,0)
    frontVisited={}
    backVisited = {}
    previous={start:None}
    next = {goal:None}
    while frontFrontier and backFrontier:
        if frontFrontier:
            frontStep +=1
            #print(frontStep)
            Fstate = frontFrontier.pop()
            #print(Fstate)
            #print(Fstate[1] == goal or backFrontier.contains(Fstate[1]))
            if Fstate[1] == goal or backFrontier.contains(Fstate[1]):
                #print(backFrontier.contains(Fstate[1]))
                if return_cost:
                    if return_nexp:
                        return path(previous,Fstate[1]),Fstate[0],nexp
                    return path(previous,Fstate[1]),Fstate[0]
                else:
                    if return_nexp:
                        return path(previous,Fstate[1]),None,nexp
                    return path(previous,Fstate[1]), path(next,Bstate[1])
            frontVisited[Fstate[1]] = pathcost(path(previous,Fstate[1]), state_graph)
            for nextState in state_graph[Fstate[1]]:
                updateCost = frontVisited[Fstate[1]]+state_graph[Fstate[1]][nextState]+forwardHeuristic(nextState)
                if( nextState not in frontVisited and nextState not in frontFrontier.states):
                    frontFrontier.add(nextState,updateCost)
                    previous[nextState] = Fstate[1]
                elif( nextState in frontFrontier.states and frontFrontier.states[nextState] > updateCost ):
                    frontFrontier.replace(nextState, updateCost)
                    previous[nextState] = Fstate[1]
        if backFrontier:
            backStep += 1
            #print(backStep)
            Bstate = backFrontier.pop()
            if Bstate[1] == start or frontFrontier.contains(Bstate[1]):
                #print("back")
                if return_cost:
                    if return_nexp:
                        return path(next,Bstate[1]),Bstate[0],nexp
                    return path(next,Bstate[1]),Bstate[0]
                else:
                    if return_nexp:
                        return path(next,Bstate[1]),None,nexp
                    return path(previous,Fstate[1]),path(next,Bstate[1])
            backVisited[Bstate[1]] = pathcost(path(next,Bstate[1]), state_graph)
            for prevState in state_graph[Bstate[1]]:
                updateCost = backVisited[Bstate[1]]+state_graph[Bstate[1]][prevState]+backwardHeuristic(nextState)
                if( prevState not in backVisited and prevState not in backFrontier.states):
                    backFrontier.add(prevState,updateCost)
                    next[prevState] = Bstate[1]
                elif( prevState in backFrontier.states and backFrontier.states[prevState] > updateCost ):
                    backFrontier.replace(prevState, updateCost)
                    next[prevState] = Bstate[1]

### Running Example

In [16]:
path1,path2 = astar_search('chi','pro', map_distances, forward_heuristic_sld_providence, backward_heuristic_sld_providence, return_cost=False, return_nexp=False)
print("Path:",path1,path2)

Path: ['chi', 'cle', 'pit'] ['pro', 'new', 'phi']


### Results

From this bidirectional search, we have proved that it is possible to short-circuit the path finding problem, essentially turning it into a meet in the middle problem. This served as a proof of concept for your omnidirectional A\*. However, this did also bring to light some situations in which our implementation of omni A* won't work (see conclusion)

# Omnidirectional A* Search
___

In [17]:
# Heuristic, in this case euclidian distance from a node to all other nodes 
small_graph_heursitic = {
    "a": {
        "a": 0,
        "b": 70,
        "c": 85,
        "d": 115,
        "e": 230,
        "f": 355,
        "g": 255
    },
    "b": {
        "a": 70,
        "b": 0,
        "c": 65,
        "d": 150,
        "e": 190,
        "f": 290,
        "g": 205
    },
    "c": {
        "a": 88,
        "b": 65,
        "c": 0,
        "d": 90,
        "e": 165,
        "f": 265,
        "g": 180
    },
    "d": {
        "a": 115,
        "b": 150,
        "c": 105,
        "d": 0,
        "e": 165,
        "f": 280,
        "g": 180
    },
    "e": {
        "a": 230,
        "b": 190,
        "c": 145,
        "d": 165,
        "e": 0,
        "f": 105,
        "g": 20
    },
    "f": {
        "a": 355,
        "b": 290,
        "c": 285,
        "d": 265,
        "e": 100,
        "f": 0,
        "g": 95
    },
    "g": {
        "a": 255,
        "b": 205,
        "c": 180,
        "d": 180,
        "e": 20,
        "f": 95,
        "g": 0
    }
}

In [18]:
# New idea: Omnidirectional A* Search, terminating when paths from all sources collide, that IS guaranteed to find 
# the optimal meeting point, defined as the point that would required the least total travel from all sources, for 
# any number (N) source nodes in a weighted, connected graph, without necessarily determining the SSSP from all 
# source node.

def meetAStar(sources, graph, heuristic):
    open_list = {}
    open_list_keys = {}
    closed_list = {}
    costs = {}
    nodes_expanded = 0
    for source in sources:
        open_list[source] = []
        total_cost = 0
        for s in sources:
            if s != source:
                total_cost += heuristic[source][s]
        heapq.heappush(open_list[source], (total_cost, source))
        open_list_keys[source] = set(source)
        closed_list[source] = set()
        costs[source] = {source: 0}
    
    while any(open_list):
        for source in sources:
            min_node = heapq.heappop(open_list[source])[1]
            open_list_keys[source].remove(min_node)
            closed_list[source].add(min_node)
            nodes_expanded += 1
            found = True
            for k in sources:
                if k != source and min_node not in closed_list[k]:
                    found = False
            if found:
                print("\n# of Nodes Expanded: %d" % (nodes_expanded))
                print("----------------------------")
                print("Meeting point:", min_node)
                total = 0
                for s in sources:
                    total += costs[s][min_node]
                    print("Distance from %s: %d" % (s, costs[s][min_node]))
                print("Total distance traveled:",total)
                return min_node
        
            for n in graph[min_node]:
                if n not in closed_list[source]:
                    cost = costs[source][min_node] + graph[min_node][n]
                    for k in sources: # may want to consider all
                        cost += heuristic[n][k]
                    if n in open_list_keys[source]:
                        for i in range(len(open_list[source])):
                            if open_list[source][i][1] == n:
                                if open_list[source][i][0] > cost:
                                    del[open_list[source][i]]
                                    open_list_keys[source].remove(n)
                                break
                    if n not in open_list_keys[source]:
                        open_list_keys[source].add(n)
                        heapq.heappush(open_list[source], (cost, n))
                        costs[source][n] = costs[source][min_node] + graph[min_node][n]        

### Running Examples:

In [19]:
# Running omnidirectional A* on 6 examples
meetAStar(["a", "d", "e"], small_graph, small_graph_heursitic)
meetAStar(["a", "f", "e"], small_graph, small_graph_heursitic)
meetAStar(["a", "c", "f"], small_graph, small_graph_heursitic)
meetAStar(["c", "g", "d"], small_graph, small_graph_heursitic)
meetAStar(["e", "f", "g"], small_graph, small_graph_heursitic)
meetAStar(["a", "b", "c", "d", "f"], small_graph, small_graph_heursitic)


# of Nodes Expanded: 6
----------------------------
Meeting point: d
Distance from a: 125
Distance from d: 0
Distance from e: 175
Total distance traveled: 300

# of Nodes Expanded: 10
----------------------------
Meeting point: e
Distance from a: 285
Distance from f: 110
Distance from e: 0
Total distance traveled: 395

# of Nodes Expanded: 12
----------------------------
Meeting point: b
Distance from a: 85
Distance from c: 70
Distance from f: 280
Total distance traveled: 435

# of Nodes Expanded: 8
----------------------------
Meeting point: d
Distance from c: 100
Distance from g: 195
Distance from d: 0
Total distance traveled: 295

# of Nodes Expanded: 5
----------------------------
Meeting point: g
Distance from e: 20
Distance from f: 100
Distance from g: 0
Total distance traveled: 120

# of Nodes Expanded: 14
----------------------------
Meeting point: b
Distance from a: 85
Distance from b: 0
Distance from c: 70
Distance from d: 170
Distance from f: 280
Total distance traveled: 60

'b'

### Results:

__Output:__ Omnidirectional A*  found the optimal meeting point and the optimal distance in all 6 of the 6 examples. 

__Performance:__ Omnidirectional A* expanded much fewer nodes than SumOfPaths in every example, making it much more efficient.